In [1]:
import tensorflow as tf
import numpy as np
from nltk.tokenize import word_tokenize
from itertools import chain
import functools
import pickle

In [2]:
def save_pickle(d, path):
    with open(path, mode='wb') as f:
        pickle.dump(d, f)
def load_pickle(path):
    with open(path, mode='rb') as f:
        return pickle.load(f)

In [3]:
# with open('data/movie_dialog_dataset/task1_qa/task1_qa_pipe_train.txt', 'r') as f:
with open('data/qa1_single-supporting-fact_train.txt', 'r') as f:
    lines = f.readlines()
    data = []
    for l in lines:
        l = l.rstrip()
        if '\t' not in l: continue
        turn, left = l.split(' ', 1)
        q, a = left.split('\t', 1)
#         data.append((word_tokenize(q), a.split('|')))
        data.append((word_tokenize(q), [a.split('\t')[0]]))
# save_pickle('data.pickle')
print(data[0])

(['Where', 'is', 'Mary', '?'], ['bathroom'])


In [4]:
max_query_size = max(map(len, chain.from_iterable(q for q, _ in data)))
vocab = sorted(functools.reduce(lambda x, y: x | y, (set(q + a) for q, a in data)))
vocab_size = len(vocab) + 1 # +1 for nil word
w2i = dict((c, i+1) for i, c in enumerate(vocab))
w2i['__NIL__'] = 0
# vocab_q = sorted(functools.reduce(lambda x, y: x | y, (set(q) for q, _ in data)))
# vocab_a = sorted(functools.reduce(lambda x, y: x | y, (set(a) for _, a in data)))
# print(len(vocab))

In [5]:
w2i_q = dict((c, i+1) for i, c in enumerate(vocab_q))
# w2i_q['__NIL__'] = 0
i2w_q = dict((i+1, c) for i, c in enumerate(vocab_q))
# i2w[0] = '__NIL__'
w2i_a = dict((c, i+1) for i, c in enumerate(vocab_a))
i2w_a = dict((i+1, c) for i, c in enumerate(vocab_a))

NameError: name 'vocab_q' is not defined

In [6]:
# save_pickle(vocab, 'vocab.pickle')

In [13]:
data[0]
len(w2i)

14

In [14]:
def vectorize(data):
    Q, A = [], []
    for question, answer in data:
        q = [w2i[w] for w in question] + [0] * max(0, max_query_size - len(question)) # padding
        
        a = np.zeros(len(w2i))
#         print(answer)
        for ans in answer:
            a[w2i[ans]] = 1
        
        Q.append(q)
        A.append(a)
    
    return np.array(Q), np.array(A)

Q, A = vectorize(data[:10000])

In [9]:
print(Q[0], A[0])
print(len(Q), len(Q[0]), len(A), len(A[0]))

[ 6 11  4  1  0  0] [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
1000 6 1000 14


In [10]:
save_pickle(Q, 'Q.pickle')
save_pickle(A, 'A.pickle')

## MLP w/ Tensorflow

In [17]:
# Parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 128
display_step = 1

# Network Parameters
n_hidden_1 = 8 # 1st layer number of neurons
n_hidden_2 = 8  # 2nd layer number of neurons
# n_hidden_3 = 1024 # 3rd layer number of neurons
n_input = max_query_size
# n_classes = 10 # MNIST total classes (0-9 digits)
n_classes = vocab_size
print('n_classes', n_classes)

# tf Graph input
X = tf.placeholder("float", [None, n_input], name="X")
Y = tf.placeholder("float", [None, n_classes], name="Y")

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
#     'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
#     'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Create model
def multilayer_perceptron(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
#     layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


# Construct model
logits = multilayer_perceptron(X)


# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

n_classes 14


In [18]:
with tf.Session() as sess:
    sess.run(init)
    n_train = len(Q)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_train/batch_size)
        # Loop over all batches
#         for i in range(total_batch):
        for start in range(0, n_train, batch_size):
            end = start + batch_size
#             batch_x, batch_y = mnist.train.next_batch(batch_size)
            batch_x, batch_y = Q[start:end], A[start:end]
    
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
            
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: Q[:100], Y: A[:100]}))

Epoch: 0001 cost=343.604357038
Epoch: 0002 cost=324.511104039
Epoch: 0003 cost=305.859436035
Epoch: 0004 cost=288.811285836
Epoch: 0005 cost=276.914372035
Epoch: 0006 cost=265.154357910
Epoch: 0007 cost=254.035014561
Epoch: 0008 cost=243.358516148
Epoch: 0009 cost=232.834365845
Epoch: 0010 cost=222.602735247
Epoch: 0011 cost=212.611934117
Epoch: 0012 cost=202.866973877
Epoch: 0013 cost=193.333971296
Epoch: 0014 cost=184.005194528
Epoch: 0015 cost=174.871239798
Epoch: 0016 cost=165.918215070
Epoch: 0017 cost=157.129418509
Epoch: 0018 cost=148.492203849
Epoch: 0019 cost=139.993629456
Epoch: 0020 cost=131.618959699
Epoch: 0021 cost=123.354365758
Epoch: 0022 cost=115.186367580
Epoch: 0023 cost=107.100591387
Epoch: 0024 cost=99.082859584
Epoch: 0025 cost=91.118700300
Epoch: 0026 cost=83.193087987
Epoch: 0027 cost=75.290742602
Epoch: 0028 cost=67.395771027
Epoch: 0029 cost=59.491915567
Epoch: 0030 cost=51.647338867
Epoch: 0031 cost=45.775518145
Epoch: 0032 cost=41.105758122
Epoch: 0033 cost=